# Single Table Models Template

In [ ]:
# Removes lint errors from VS Code
from typing import Dict, TYPE_CHECKING, Tuple, List

if TYPE_CHECKING:
    import kedro

    catalog: kedro.io.data_catalog.DataCatalog
    session: kedro.framework.session.session.KedroSession
    pipelines: Dict[str, kedro.pipeline.pipeline.Pipeline]

In [ ]:
import os

VIEW = os.getenv("DATASET_VIEW") or "tab_adult"
TABLE = os.getenv("DATASET_TABLE") or "table"
ALG = os.getenv("SYNTH_ALG") or "ref"

import numpy as np
import pandas as pd

wrk: pd.DataFrame = catalog.load(f"{VIEW}.wrk.{TABLE}")
alg: pd.DataFrame = catalog.load(f"{VIEW}.{ALG}.{TABLE}")
tst: pd.DataFrame = catalog.load(f"{VIEW}.dev.{TABLE}")

from pasteur.metadata import Metadata
meta = catalog.load(f"params:{VIEW}.metadata")
meta = Metadata(meta, wrk).get_table(TABLE)

NameError: name 'catalog' is not defined

In [ ]:
wrk.head()

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
1,6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica
2,7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States
3,9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States
4,10,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States


In [ ]:
wrk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13024 entries, 0 to 13023
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   id              13024 non-null  int64   
 1   age             13024 non-null  int32   
 2   workclass       13024 non-null  category
 3   fnlwgt          13024 non-null  int32   
 4   education       13024 non-null  category
 5   education-num   13024 non-null  int32   
 6   marital-status  13024 non-null  category
 7   occupation      13024 non-null  category
 8   relationship    13024 non-null  category
 9   race            13024 non-null  category
 10  sex             13024 non-null  category
 11  capital-gain    13024 non-null  int32   
 12  capital-loss    13024 non-null  int32   
 13  hours-per-week  13024 non-null  int32   
 14  native-country  13024 non-null  category
dtypes: category(8), int32(6), int64(1)
memory usage: 614.5 KB
